In [9]:
import neuralnet
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.metrics import precision_score, confusion_matrix

In [10]:
data = pd.read_csv('../data.csv')
# 将带有中文属性的词转换为数值
print(data)
listUniq = data.ix[:, 'drill'].unique()
for j in range(len(listUniq)):
    data.ix[:, 'drill'] = data.ix[:, 'drill'].apply(
        lambda x: j if x == listUniq[j] else x)
listUniq = data.ix[:, 'rock'].unique()
for j in range(len(listUniq)):
    data.ix[:, 'rock'] = data.ix[:, 'rock'].apply(
        lambda x: j if x == listUniq[j] else x)

     kong         drill    D     N  ROM    L     V rock
0       1  (1,0,0,0,0)   110   200  150  250  3.63   砂岩
1       1  (0,1,0,0,0)   110   200  150  250  4.50   砂岩
2       1  (1,0,0,0,0)   110   200  150  250  3.08   砂岩
3       1  (1,0,0,0,0)   110   200  150  250  2.38   泥岩
4       1  (1,0,0,0,0)   110   200  150  250  0.89   砂岩
..    ...           ...  ...   ...  ...  ...   ...  ...
199     7  (0,0,1,0,0)    73  1000  150  250  4.53   砂岩
200     7  (0,0,0,1,0)    89  1000  150  250  2.10   煤岩
201     7  (0,0,1,0,0)    91  1000  150  250  3.06   泥岩
202     7  (0,0,1,0,0)    91  1000  150  250  0.92   灰岩
203     7  (0,0,1,0,0)    91  1000  150  250  3.32   泥岩

[204 rows x 8 columns]


d:\env\pytorch\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  after removing the cwd from sys.path.
d:\env\pytorch\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  
d:\env\pytorch\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  import sys
d:\env\pytorch\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: 


In [11]:
# 进行one-hot编码
tempdata = data[['drill']]
enc = preprocessing.OneHotEncoder()
enc.fit(tempdata)
# 将结果转换为二维数组
tempdata = enc.transform(tempdata).toarray()
# 再将二维数组转换为DataFrame，记得这里会变成多列
tempdata = pd.DataFrame(tempdata, columns=['drill'] * len(tempdata[0]))

d:\env\pytorch\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [12]:
# 将输出转换为one-hot编码
rockdata = data[['rock']]
enc_rock = preprocessing.OneHotEncoder()
enc_rock.fit(rockdata)
rockdata = enc_rock.transform(rockdata).toarray()

# 将孔洞转换为one-hot编码
kongdata = data[['kong']]
enc_kong = preprocessing.OneHotEncoder()
enc_kong.fit(kongdata)
kongdata = enc_kong.transform(kongdata).toarray()
kongdata = pd.DataFrame(kongdata, columns=['kong'] * len(kongdata[0]))

d:\env\pytorch\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
d:\env\pytorch\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder be

In [17]:
# 将其他数据读取到x_columns中
x_columns = [x for x in data.columns if x in ['D', 'N', 'L', 'V', 'ROM']]
# x = pd.concat([kongdata, tempdata, data[x_columns]],  axis=1)
# 将探头数据与其他数据合并作为输入
x = pd.concat([tempdata, data[x_columns]], axis=1)
X = np.array(x)
# np.random.shuffle(X)
# 归一化
X = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
# X = X.transpose()
print(x)

     drill  drill  drill  drill  drill    D     N  ROM    L
0      1.0    0.0    0.0    0.0    0.0  110   200  150  250
1      0.0    1.0    0.0    0.0    0.0  110   200  150  250
2      1.0    0.0    0.0    0.0    0.0  110   200  150  250
3      1.0    0.0    0.0    0.0    0.0  110   200  150  250
4      1.0    0.0    0.0    0.0    0.0  110   200  150  250
..     ...    ...    ...    ...    ...  ...   ...  ...  ...
199    0.0    0.0    1.0    0.0    0.0   73  1000  150  250
200    0.0    0.0    0.0    0.0    1.0   89  1000  150  250
201    0.0    0.0    1.0    0.0    0.0   91  1000  150  250
202    0.0    0.0    1.0    0.0    0.0   91  1000  150  250
203    0.0    0.0    1.0    0.0    0.0   91  1000  150  250

[204 rows x 9 columns]


In [8]:
import random
# Y为一个(204, )的数组，y为（204， 4）的数组
Y = data['rock']
y = np.array(rockdata)
# select = np.random.randint(0, 204, size=[40,])
select = random.sample(range(0, X.shape[0]), 40)
select = np.sort(select)
train_select = []
for i in range(0, X.shape[0]):
    if i not in select:
        train_select.append(i)
train_select = np.array(train_select)
x_test = X[select, :]
y_test = y[select, :]
x_train = X[train_select, :]
y_train = y[train_select, :]
Y_train = y_train.argmax(axis=1)
Y_test = y_test.argmax(axis=1)

In [ ]:
# 输入数据进行训练
train = neuralnet.train(x_train, y_train, hiddenNum=3, unitNum=18, alpha=0.1, maxIters=1000000, precision=0.01)
# 读取神经网络的参数
Thetas = train['Thetas']
# 获得预测值
train_predictions = neuralnet.predict(x_train, Thetas)
train_predictions = train_predictions.transpose()
train_predict = train_predictions.argmax(axis=1)
print(train_predict)

Doing Gradient Checking....
第0次数损失为2.9236836151240606
第1000次数损失为1.8306691402245614
第2000次数损失为1.7927520787910995
第3000次数损失为1.7846798764421512
第4000次数损失为1.7791555518684652
第5000次数损失为1.7735911946843335
第6000次数损失为1.7677021115585363
第7000次数损失为1.761435518609078
第8000次数损失为1.754710535139942
第9000次数损失为1.7474054068233587
第10000次数损失为1.7392497636783626
第11000次数损失为1.7299309555165348
第12000次数损失为1.7192026755551275
第13000次数损失为1.706784661470073
第14000次数损失为1.692117322177353
第15000次数损失为1.6741359080883447
第16000次数损失为1.6517161896098729
第17000次数损失为1.625583730855921
第18000次数损失为1.597969126135805
第19000次数损失为1.570889516062925
第20000次数损失为1.5469915559891851
第21000次数损失为1.526470568064251
第22000次数损失为1.5074477897837562
第23000次数损失为1.4876019156934117
第24000次数损失为1.4649241580063224
第25000次数损失为1.4391175979666884
第26000次数损失为1.4102176994680586
第27000次数损失为1.3780924919258466
第28000次数损失为1.3472849060019343
第29000次数损失为1.3317936086341575
第30000次数损失为1.305540338191757
第31000次数损失为1.2801070712932732
第32000次数损失为1.2553380120367699
第330

In [ ]:
train_accuracy = precision_score(Y_train, train_predict, average='micro')*100
print("训练集准确率：{} %" .format(train_accuracy))
train_matrix = confusion_matrix(Y_train, train_predict)
print(train_matrix)

In [ ]:
# 获得预测值
test_predictions = neuralnet.predict(x_test, Thetas)
test_predictions = test_predictions.transpose()
test_predict = test_predictions.argmax(axis=1)
test_accuracy = precision_score(Y_test, test_predict, average='micro')*100
print("训练集准确率：{} %" .format(test_accuracy))
test_matrix = confusion_matrix(Y_test, test_predict)
print(test_matrix)